In [1]:
import numpy as np
import pandas as pd
import gurobipy as gp

In [32]:
# Read population data
pop = pd.read_csv('./nys_pop_by_zipcode_age/population.csv')

In [33]:
pop["0-14"] = pop["-5"] + pop["5-9"] + pop["10-14"]
pop = pop[["zipcode", "0-14"]]
# change zipcode to string
pop["zipcode"] = pop["zipcode"].astype(str)

In [34]:
pop

,zipcode,0-14
0,6390,6
1,10001,2470
2,10002,8386
3,10003,3427
4,10004,776
...,...,...
1641,14774,76
1642,14785,0
1643,14788,1
1644,14805,76


In [24]:
# Read population data
child_care = pd.read_csv('./child_care_regulated/child_care_regulated.csv')


In [25]:
child_care = child_care[['zip_code', 'total_capacity']]
child_care['zip_code'] = child_care['zip_code'].map(lambda x: str(x)[:5])
child_care = child_care.groupby('zip_code').sum().reset_index()

In [26]:
child_care

,zip_code,total_capacity
0,10001,609
1,10002,4684
2,10003,1995
3,10004,263
4,10005,39
...,...,...
1183,14897,16
1184,14901,709
1185,14903,97
1186,14904,258


In [40]:
# Merge data
data = pd.merge(pop, child_care, left_on='zipcode', right_on='zip_code', how='left')
data = data[["zipcode", "0-14", "total_capacity" ]]
data = data.fillna(0)
data["total_capacity"] = data["total_capacity"].astype(int)
data

,zipcode,0-14,total_capacity
0,6390,6,0
1,10001,2470,609
2,10002,8386,4684
3,10003,3427,1995
4,10004,776,263
...,...,...,...
1641,14774,76,62
1642,14785,0,0
1643,14788,1,0
1644,14805,76,8


In [62]:
c_new = 255000
s_new = 30
c_slot = 900

In [63]:
arr_exist = data["total_capacity"].values
arr_pop = data["0-14"].values

In [64]:
# Create a new model
m = gp.Model("child_care")

# Add variables
x = m.addMVar(shape=len(arr_exist), vtype=gp.GRB.INTEGER, name="n_ext")
y = m.addMVar(shape=len(arr_exist), vtype=gp.GRB.INTEGER, name="n_new")

# Set objective
m.setObjective(c_slot * gp.quicksum(x[i] for i in range(len(arr_exist))) + c_new * gp.quicksum(y[i] for i in range(len(arr_exist))), gp.GRB.MINIMIZE)

# Add constraints
for i in range(len(arr_exist)):
    m.addConstr(x[i] <= 0.5 * arr_exist[i])
    m.addConstr(3 * (s_new * y[i] + x[i]) >= arr_pop[i])

m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 3292 rows, 3292 columns and 4938 nonzeros
Model fingerprint: 0x7471787a
Variable types: 0 continuous, 3292 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 9e+01]
  Objective range  [9e+02, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+04]
Found heuristic solution: objective 9.435000e+09
Presolve removed 3291 rows and 3290 columns
Presolve time: 0.24s
Presolved: 1 rows, 2 columns, 2 nonzeros
Found heuristic solution: objective 7.053311e+09
Variable types: 0 continuous, 2 integer (0 binary)

Explored 0 nodes (0 simplex iterations) in 0.25 seconds (0.01 work units)
Thread count was 16 (of 16 available processors)

Solution count 2: 7.05331e+09 9.435e+09 

Optimal s

In [66]:
# Print solution
for i in range(len(arr_exist)):
    print(f"Zipcode {data['zipcode'][i]}: {int(x[i].x)} existing slots, {int(y[i].x)} new facilities")

print()
print(f"Total cost: {m.objVal}")

Zipcode 6390: 0 existing slots, 1 new facilities
Zipcode 10001: 284 existing slots, 18 new facilities
Zipcode 10002: 2316 existing slots, 16 new facilities
Zipcode 10003: 993 existing slots, 5 new facilities
Zipcode 10004: 109 existing slots, 5 new facilities
Zipcode 10005: 6 existing slots, 10 new facilities
Zipcode 10006: 70 existing slots, 1 new facilities
Zipcode 10007: 119 existing slots, 11 new facilities
Zipcode 10009: 867 existing slots, 31 new facilities
Zipcode 10010: 91 existing slots, 41 new facilities
Zipcode 10011: 958 existing slots, 10 new facilities
Zipcode 10012: 4 existing slots, 13 new facilities
Zipcode 10013: 195 existing slots, 35 new facilities
Zipcode 10014: 562 existing slots, 2 new facilities
Zipcode 10016: 410 existing slots, 36 new facilities
Zipcode 10017: 51 existing slots, 5 new facilities
Zipcode 10018: 75 existing slots, 4 new facilities
Zipcode 10019: 307 existing slots, 15 new facilities
Zipcode 10020: 0 existing slots, 0 new facilities
Zipcode 10021

In [68]:
# Problem B

B = 1e10

In [69]:
# Create a new model
m = gp.Model("child_care")

# Add variables
x = m.addMVar(shape=len(arr_exist), vtype=gp.GRB.INTEGER, name="n_ext")
y = m.addMVar(shape=len(arr_exist), vtype=gp.GRB.INTEGER, name="n_new")

# Set objective
m.setObjective(s_new * gp.quicksum(y[i] for i in range(len(arr_exist))) + gp.quicksum(x[i] for i in range(len(arr_exist))), gp.GRB.MAXIMIZE)

# Add constraints
for i in range(len(arr_exist)):
    m.addConstr(x[i] <= 0.5 * arr_exist[i])
    m.addConstr(3 * (s_new * y[i] + x[i]) >= arr_pop[i])
    m.addConstr(c_slot * gp.quicksum(x[i] for i in range(len(arr_exist))) + c_new * gp.quicksum(y[i] for i in range(len(arr_exist))) <= B)
# needs to be changed (subject to the total population constraint)

m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 11+.0 (22631.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-11800H @ 2.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 4938 rows, 3292 columns and 5423570 nonzeros
Model fingerprint: 0x7cee2a43
Variable types: 0 continuous, 3292 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+05]
  Objective range  [1e+00, 3e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+10]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 1456777.0000
Presolve removed 139 rows and 0 columns (presolve time = 236s) ...
Presolve removed 2399 rows and 1504 columns (presolve time = 240s) ...
Presolve removed 4044 rows and 1504 columns
Presolve time: 241.33s
Presolved: 894 rows, 1788 columns, 3574 nonzeros
Found heuristic s

In [70]:
# Print solution
for i in range(len(arr_exist)):
    print(f"Zipcode {data['zipcode'][i]}: {int(x[i].x)} existing slots, {int(y[i].x)} new facilities")

print()
print(f"Total slots: {sum(x[i].x for i in range(len(arr_exist))) + s_new * sum(y[i].x for i in range(len(arr_exist)))}")

Zipcode 6390: 0 existing slots, 1 new facilities
Zipcode 10001: 304 existing slots, 6428 new facilities
Zipcode 10002: 2342 existing slots, 16 new facilities
Zipcode 10003: 997 existing slots, 39 new facilities
Zipcode 10004: 131 existing slots, 5 new facilities
Zipcode 10005: 19 existing slots, 10 new facilities
Zipcode 10006: 78 existing slots, 1 new facilities
Zipcode 10007: 142 existing slots, 11 new facilities
Zipcode 10009: 892 existing slots, 31 new facilities
Zipcode 10010: 117 existing slots, 41 new facilities
Zipcode 10011: 978 existing slots, 42 new facilities
Zipcode 10012: 12 existing slots, 14 new facilities
Zipcode 10013: 217 existing slots, 35 new facilities
Zipcode 10014: 568 existing slots, 21 new facilities
Zipcode 10016: 411 existing slots, 50 new facilities
Zipcode 10017: 53 existing slots, 7 new facilities
Zipcode 10018: 87 existing slots, 4 new facilities
Zipcode 10019: 313 existing slots, 26 new facilities
Zipcode 10020: 22 existing slots, 0 new facilities
Zipco